In [1]:
#create some t-SNE examples using the MNIST and digits datasets
#https://medium.com/@violante.andre/an-introduction-to-t-sne-with-python-example-47e6ae7dc58f

In [3]:
!pip install swat

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.0/465.0 kB 5.4 MB/s eta 0:00:00a 0:00:01


In [4]:
# Load Python Libraries
import swat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
%matplotlib inline

In [6]:
# Create CAS Connection
#conn = swat.CAS(host, portnum, protocol='http')
#conn.sessionProp.setSessOpt(messageLevel='NONE'); # Suppress CAS Messages

In [7]:
mnist_train = pd.read_csv('/Users/jacksonwalters/Documents/GitHub/t-SNE_examples/mnist/mnist_train.csv')
mnist_test = pd.read_csv('/Users/jacksonwalters/Documents/GitHub/t-SNE_examples/mnist/mnist_test.csv')

In [13]:
#the digits are 28x28 = 784 pixels, which is the number of columns
#there are 60,000 labeled examples in the training data set, and 10,000 labeled examples in the test dataset

display(mnist_train.shape)
display(mnist_test.shape)

(60000, 785)

(10000, 785)

In [8]:
mnist_train.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
mnist_test

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# View Label Distribution
# Digits Labels: 0-5, MNIST Labels: 0-10
display(pd.crosstab(index=mnist_train['label'], columns='count'))
display(pd.crosstab(index=mnist_test['label'], columns='count'))

col_0,count
label,
0,5923
1,6742
2,5958
3,6131
4,5842
5,5421
6,5918
7,6265
8,5851


col_0,count
label,
0,980
1,1135
2,1032
3,1010
4,982
5,892
6,958
7,1028
8,974
